In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from datetime import date
import numpy as np
import os
import pandas as pd

from summer.utils import ref_times_to_dti

from autumn.core.inputs.mobility.queries import get_mobility_data
from autumn.core.inputs.database import get_input_db

from autumn.settings import Region, Models
from autumn.settings.constants import COVID_BASE_DATETIME

from autumn.models.sm_sir.mixing_matrix.macrodistancing import weight_mobility_data


In [ ]:
model = Models.SM_SIR
region = Region.NCR

project = get_project(model, region)
params = project.param_set.baseline

In [ ]:
print("Whether the mobility effects are actually turned on at all:")
project.param_set.baseline["is_dynamic_mixing_matrix"]

In [ ]:
y_upper = 2.

# Collate data together
input_db = get_input_db()

mob_df, int_times = get_mobility_data(iso3, pop_region, COVID_BASE_DATETIME)
google_mob_df = weight_mobility_data(mob_df, project.param_set.baseline["mobility"]["google_mobility_locations"])

times = ref_times_to_dti(COVID_BASE_DATETIME, int_times)

# Get plots ready
mob_fig, mob_axes = plt.subplots(1, 2, figsize=(12, 6))
plot_left_date = date(2020, 1, 1)
plot_right_date = times[-1]  # Not sure why this is necessary

# Plot raw mobility data
ax = mob_axes[0]
for mobility_domain in ["grocery_and_pharmacy", "residential", "parks", "retail_and_recreation", "transit_stations"]:
    ax.plot(times, mob_df[mobility_domain], label=mobility_domain)
ax.set_ylim((0., y_upper))
ax.tick_params(axis="x", labelrotation=45)
ax.set_title("raw Google mobility domains")
ax.legend(loc="lower right")
ax.set_xlim(left=plot_left_date, right=plot_right_date)

# Plot processed mobility data
ax = mob_axes[1]
for location in list(project.param_set.baseline["mobility"]["google_mobility_locations"].keys()):
    ax.plot(times, google_mob_df[location], label=location)
ax.tick_params(axis="x", labelrotation=45)
ax.set_ylim((0., y_upper))
ax.legend(loc="lower left")
ax.set_title("mobility as implemented in the model")
mob_fig.tight_layout(w_pad=1.5, h_pad=3.5)
ax.set_xlim(left=plot_left_date, right=plot_right_date)